In [1]:
#  run hmmratac on filtered bam of any length

In [2]:
import os 
import subprocess as sp
import pandas as pd
import glob
from multiprocessing import Pool
import pybedtools
import importlib
pybedtools.set_bedtools_path('/home/rabbani/localenv/anaconda/miniconda3/envs/matplotlib/bin/')
pybedtools = importlib.reload(pybedtools)

In [3]:
root = os.path.join("/data/manke/group/rabbani/atac_project2078")
output_path = os.path.join("/data/akhtar/group2/rabbani/atac_project2078/")

In [4]:
# sort and index bam files: RUN IT ONLY ONCE!

In [5]:
for file in glob.glob(os.path.join(root,"snakepipes_output/filtered_bam/", "*.bam")):
    name = os.path.basename(file)
    cmd = "module load samtools; samtools sort -@ 14 "+file
    cmd += " -o "+os.path.join(root, "hmmratac/filteredbams_sorted", name.split(".bam")[0]+".sorted.bam")
    cmd += "; samtools index -@ 10 "+os.path.join(root, "hmmratac/filteredbams_sorted",
                                            name.split(".bam")[0]+".sorted.bam")
    sp.check_output(cmd, shell = True)

In [6]:
# run peak calling on sorted bams RUN IT ONLY ONCE!
def peak_calling(file):
    name = os.path.basename(file).split(".sorted.bam")[0]
    cmd = "module load  slurm/19.05.3-2; SlurmEasy -t 15 \'"
    cmd += "java -jar /data/processing1/leily/hmmratac/HMMRATAC_V1.2.10_exe.jar -b "+file+" -i "+file+".bai "
    cmd += " -g /data/repository/organisms/GRCm38_ensembl/genome_fasta/genome.chrom.sizes "
    cmd += " --blacklist /data/repository/organisms/GRCm38_ensembl/akundaje/blacklist.bed "
    cmd += " -f False -m 50.0,157.25,354.26,712.46 -s 20.0,40.21,79.91,119.62 "
    cmd += " --bedgraph True --bgscore True "
    cmd += " -o "+os.path.join(output_path,"hmmratac/peaks", name)+"\'"
    sp.check_output(cmd, shell = True)
with Pool(9) as p:
    p.map(peak_calling, glob.glob(os.path.join(root,"hmmratac/filteredbams_sorted", "*.bam")))
for file in glob.glob(os.path.join(root,"hmmratac/filteredbams_sorted", "V*.bam")):
    peak_calling(file)

In [7]:
# filter the ocr part of peaks & remove hiCov regions to use them for the diff analysis

In [8]:
union_df = pd.DataFrame()
for file in glob.glob(os.path.join(root, "hmmratac", "peaks", "*.gappedPeak")):
    name = os.path.basename(file).split("gappedPeak")[0]
    print(name)
    df = pd.read_csv(file, sep = "\t", header = None)
    df = df.loc[~ df[3].str.contains("HighCoveragePeak_")]   
    df = df[[0,6,7,12]]
    df.to_csv(os.path.join(root, "hmmratac", "ocrs", name+"bed"), header = None, index =False, sep = "\t")
    print("hmmratac:", len(df))
    macs_file = "/data/manke/group/rabbani/atac_project2078/atac_pipeline/MACS2/"+name+"narrowPeak"
    macs_file = macs_file.replace("-", "_")
    print("macs2:", len(pd.read_csv(macs_file)))
    if len(union_df) == 0:
        union_df = df[[0,6,7]]
    else:
        union_df = pd.concat([union_df, df[[0,6,7]]], ignore_index = True)
print(len(union_df))
a = pybedtools.BedTool.from_dataframe(union_df)
a = a.sort()
a = a.merge()
a.saveas(os.path.join(root, "hmmratac","unionOfPeaks.bed"))

Vector-LPS12-rep1.filtered_peaks.
hmmratac: 70576
macs2: 127563
Vector-LPS12-rep2.filtered_peaks.
hmmratac: 80082
macs2: 140249
K197R-LPS12-rep1.filtered_peaks.
hmmratac: 70397
macs2: 126801
K197R-LPS12-rep2.filtered_peaks.
hmmratac: 64216
macs2: 120441
Vector-LPS0-rep1.filtered_peaks.
hmmratac: 51652
macs2: 111328
WT-LPS12-rep3.filtered_peaks.
hmmratac: 68300
macs2: 123504
K197Q-LPS12-rep2.filtered_peaks.
hmmratac: 65788
macs2: 134963
WT-LPS3-rep3.filtered_peaks.
hmmratac: 65121
macs2: 117662
K197R-LPS3-rep3.filtered_peaks.
hmmratac: 52351
macs2: 96229
Vector-LPS0-rep2.filtered_peaks.
hmmratac: 53504
macs2: 101892
K197Q-LPS12-rep1.filtered_peaks.
hmmratac: 71612
macs2: 130551
K197Q-LPS0-rep3.filtered_peaks.
hmmratac: 53988
macs2: 104164
K197R-LPS0-rep1.filtered_peaks.
hmmratac: 50168
macs2: 94351
WT-LPS0-rep1.filtered_peaks.
hmmratac: 58691
macs2: 107553
K197Q-LPS3-rep1.filtered_peaks.
hmmratac: 64186
macs2: 121925
WT-LPS0-rep2.filtered_peaks.
hmmratac: 45523
macs2: 87451
Vector-LPS3-

In [9]:
# merge reps of ocrs
for cond in ['WT', 'Vector', 'K197Q', 'K197R']:
    for time in ['LPS0', 'LPS3', 'LPS12']:
        files = glob.glob(os.path.join(root,  "hmmratac", "ocrs", "-".join([cond, time, '*'])))
        cmd = "module load bedtools2/v2.27.0; cat "
        cmd += " ".join(files)
        cmd += "| bedtools sort -i stdin | bedtools merge -i stdin > "
        cmd += os.path.join(output_path, "hmmratac", "ocrs", "merged_peaks", "_".join(["merged", cond, time, '.bed']))
        print(cmd)
        sp.check_output(cmd, shell =True)

module load bedtools2/v2.27.0; cat /data/manke/group/rabbani/atac_project2078/hmmratac/ocrs/WT-LPS0-rep1.filtered_peaks.bed /data/manke/group/rabbani/atac_project2078/hmmratac/ocrs/WT-LPS0-rep3.filtered_peaks.bed /data/manke/group/rabbani/atac_project2078/hmmratac/ocrs/WT-LPS0-rep2.filtered_peaks.bed| bedtools sort -i stdin | bedtools merge -i stdin > /data/akhtar/group2/rabbani/atac_project2078/hmmratac/ocrs/merged_peaks/merged_WT_LPS0_.bed
module load bedtools2/v2.27.0; cat /data/manke/group/rabbani/atac_project2078/hmmratac/ocrs/WT-LPS3-rep3.filtered_peaks.bed /data/manke/group/rabbani/atac_project2078/hmmratac/ocrs/WT-LPS3-rep1.filtered_peaks.bed /data/manke/group/rabbani/atac_project2078/hmmratac/ocrs/WT-LPS3-rep2.filtered_peaks.bed| bedtools sort -i stdin | bedtools merge -i stdin > /data/akhtar/group2/rabbani/atac_project2078/hmmratac/ocrs/merged_peaks/merged_WT_LPS3_.bed
module load bedtools2/v2.27.0; cat /data/manke/group/rabbani/atac_project2078/hmmratac/ocrs/WT-LPS12-rep2.fi

In [10]:
# compute the count matrix over the union of all peaks (HighCoveragePeak are getting excluded)

In [11]:
cmd = "module load deeptools/3.5.0 slurm/19.05.3-2; SlurmEasy -t 10 -n 'bamSummORCs' '"
cmd += "multiBamSummary BED-file --BED "+os.path.join(root, "hmmratac","unionOfPeaks.bed") 
cmd += " -b "+" ".join(glob.glob(os.path.join(root, "hmmratac", "filteredbams_sorted", "*.bam")))
cmd += " -bl /data/repository/organisms/GRCm38_ensembl/akundaje/blacklist.bed -p 20 --outRawCounts "
cmd += os.path.join(root, "hmmratac", "multibamsummary_counts.tsv")
cmd += " -o "+os.path.join(root, "hmmratac", "multibamsummary_counts.npz")+"'"
print(cmd)
sp.check_output(cmd, shell =True)

cmd = "module load deeptools/3.5.0 slurm/19.05.3-2; SlurmEasy -t 10 -n 'bwSummORCs' '"
cmd += "multiBigwigSummary BED-file --BED "+os.path.join(root, "hmmratac","unionOfPeaks.bed") 
cmd += " -b "+" ".join(glob.glob(os.path.join(root, "atac_pipeline", "BigWigs", "*.filtered.bw")))
cmd += " -bl /data/repository/organisms/GRCm38_ensembl/akundaje/blacklist.bed -p 20 "
cmd += " -o "+os.path.join(root, "hmmratac","multibwsummary_counts_on_ocrs.npz")+"'"
print(cmd)
sp.check_output(cmd, shell =True)

module load deeptools/3.5.0 slurm/19.05.3-2; SlurmEasy -t 10 -n 'bamSummORCs' 'multiBamSummary BED-file --BED /data/manke/group/rabbani/atac_project2078/hmmratac/unionOfPeaks.bed -b /data/manke/group/rabbani/atac_project2078/hmmratac/filteredbams_sorted/WT-LPS0-rep3.filtered.sorted.bam /data/manke/group/rabbani/atac_project2078/hmmratac/filteredbams_sorted/K197R-LPS0-rep1.filtered.sorted.bam /data/manke/group/rabbani/atac_project2078/hmmratac/filteredbams_sorted/Vector-LPS12-rep2.filtered.sorted.bam /data/manke/group/rabbani/atac_project2078/hmmratac/filteredbams_sorted/WT-LPS3-rep3.filtered.sorted.bam /data/manke/group/rabbani/atac_project2078/hmmratac/filteredbams_sorted/K197R-LPS3-rep1.filtered.sorted.bam /data/manke/group/rabbani/atac_project2078/hmmratac/filteredbams_sorted/K197R-LPS12-rep2.filtered.sorted.bam /data/manke/group/rabbani/atac_project2078/hmmratac/filteredbams_sorted/Vector-LPS3-rep3.filtered.sorted.bam /data/manke/group/rabbani/atac_project2078/hmmratac/filteredbams

In [12]:
# extract high coverage regions & add them to the union peaks

In [13]:
all_peaks = union_df.copy()
all_peaks.rename(columns = {6:1, 7:2}, inplace = True)
hiCov = pd.DataFrame()
# print(len(union_df))
for file in glob.glob(os.path.join(root, "hmmratac", "peaks", "*.gappedPeak")):
    name = os.path.basename(file).split("gappedPeak")[0]
    df = pd.read_csv(file, sep = "\t", header = None)
    df = df.loc[df[3].str.contains("HighCoveragePeak_")]
#     print(len(df))
    all_peaks = pd.concat([all_peaks, df[[0,1,2]]])
    if len(hiCov) == 0:
        hiCov = df[[0,1,2]]
    else:
        hiCov = pd.concat([hiCov, df[[0,1,2]]])
    df.to_csv(os.path.join(root, "hmmratac", "hiCov", name+"bed"), columns = [0,1,2], index = False, header = False)
print(all_peaks.head())
a = pybedtools.BedTool.from_dataframe(all_peaks)
a = a.sort()
a = a.merge()
a.saveas(os.path.join(root, "hmmratac","unionOfPeaksWithHiCov.bed"))

a = pybedtools.BedTool.from_dataframe(hiCov)
a = a.sort()
a = a.merge()
a.saveas(os.path.join(root, "hmmratac","hiCovOnly.bed"))

            0       1       2
0  GL456233.1   38590   39490
1  GL456233.1  138680  139300
2  GL456233.1  159520  160050
3  GL456216.1   13760   14020
4  GL456216.1   15620   17480


In [14]:
# # matrix with all_peaks
# cmd = "module load deeptools slurm; SlurmEasy -t 10 -n 'bamSumHiCov' '"
# cmd += "multiBamSummary BED-file --BED "+os.path.join(root, "hmmratac","unionOfPeaksWithHiCov.bed") 
# cmd += " -b "+" ".join(glob.glob(os.path.join(root, "hmmratac", "filteredbams_sorted", "*.bam")))
# cmd += " -bl /data/repository/organisms/GRCm38_ensembl/akundaje/blacklist.bed -p 20 --outRawCounts "
# cmd += os.path.join(root, "hmmratac", "multibamsummary_counts_withHiCov.tsv")
# cmd += " -o "+os.path.join(root, "hmmratac", "multibamsummary_counts_withHiCov.npz")+"'"
# print(cmd)
# sp.check_output(cmd, shell =True)

cmd = "module load deeptools/3.5.0 slurm/19.05.3-2; SlurmEasy -t 10 -n 'bwSumHiCov' '"
cmd += "multiBigwigSummary BED-file --BED "+os.path.join(root, "hmmratac","unionOfPeaksWithHiCov.bed") 
cmd += " -b "+" ".join(glob.glob(os.path.join(root, "atac_pipeline", "BigWigs", "*.filtered.bw")))
cmd += " -bl /data/repository/organisms/GRCm38_ensembl/akundaje/blacklist.bed -p 20 "
cmd += " -o "+os.path.join(root, "hmmratac","multibwsummary_counts_on_withHiCov.npz")+"'"
print(cmd)
sp.check_output(cmd, shell =True)

module load deeptools/3.5.0 slurm/19.05.3-2; SlurmEasy -t 10 -n 'bwSumHiCov' 'multiBigwigSummary BED-file --BED /data/manke/group/rabbani/atac_project2078/hmmratac/unionOfPeaksWithHiCov.bed -b /data/manke/group/rabbani/atac_project2078/atac_pipeline/BigWigs/Vector_LPS3_rep1.filtered.bw /data/manke/group/rabbani/atac_project2078/atac_pipeline/BigWigs/K197R_LPS3_rep3.filtered.bw /data/manke/group/rabbani/atac_project2078/atac_pipeline/BigWigs/K197R_LPS12_rep3.filtered.bw /data/manke/group/rabbani/atac_project2078/atac_pipeline/BigWigs/Vector_LPS12_rep2.filtered.bw /data/manke/group/rabbani/atac_project2078/atac_pipeline/BigWigs/Vector_LPS0_rep3.filtered.bw /data/manke/group/rabbani/atac_project2078/atac_pipeline/BigWigs/K197R_LPS0_rep1.filtered.bw /data/manke/group/rabbani/atac_project2078/atac_pipeline/BigWigs/K197Q_LPS3_rep2.filtered.bw /data/manke/group/rabbani/atac_project2078/atac_pipeline/BigWigs/K197Q_LPS0_rep2.filtered.bw /data/manke/group/rabbani/atac_project2078/atac_pipeline/B

In [15]:
#  The next 3 cells can be run only when the count matrices are ready
# Remove quotes placed by multiBamSummary
cmd = "sed -i \"s/'//g\" "+os.path.join(root, "hmmratac", "multibamsummary_counts.tsv")
print(cmd)
sp.check_output(cmd, shell =True)

sed -i "s/'//g" /data/manke/group/rabbani/atac_project2078/hmmratac/multibamsummary_counts.tsv


In [16]:
# Remove .bam postfix
cmd = "sed -i \"s/.filtered.sorted.bam//g\" "+os.path.join(root, "hmmratac", "multibamsummary_counts.tsv")
print(cmd)
sp.check_output(cmd, shell =True)

sed -i "s/.filtered.sorted.bam//g" /data/manke/group/rabbani/atac_project2078/hmmratac/multibamsummary_counts.tsv


In [17]:
cmd = "sed -i \"s/-/_/g\" "+os.path.join(root, "hmmratac", "multibamsummary_counts.tsv")
print(cmd)
sp.check_output(cmd, shell =True)

sed -i "s/-/_/g" /data/manke/group/rabbani/atac_project2078/hmmratac/multibamsummary_counts.tsv


In [18]:
# find the missing reagions on multibamsummary (check for the lenght, coverage and balck list overlap)
df1 = pd.read_csv(os.path.join(root, "hmmratac", "multibamsummary_counts.tsv"), sep = "\t")
df1['coord'] = df1[["#chr", "start", "end"]].apply( lambda x: '_'.join(x.dropna().astype(str)), axis=1)

df2 = pd.read_csv(os.path.join(root, "hmmratac", "unionOfPeaks.bed"), sep = "\t", header = None)
df2['coord'] = df2[[0,1,2]].apply( lambda x: '_'.join(x.dropna().astype(str)), axis=1)

df_all = df1.merge(df2.drop_duplicates(), on=['coord'], how='right', indicator=True)

diff_df = df_all[df_all['_merge'] == 'right_only'][[0,1,2]]
diff_df['diff'] = diff_df[2] - diff_df[1]
diff_df['diff']

347       260
1339     1380
2590      940
2591     1140
6636      470
         ... 
96501    2050
96502    4781
96503    1475
96504     520
96505     709
Name: diff, Length: 89, dtype: int64

In [19]:
max(diff_df['diff'])

10120

In [20]:
# tobias_bigwig correction module TODO